In [ ]:
hl = .88*np.max(model)/2
psf_mask = model>=hl

plt.imshow(psf_mask,origin='lower')
plt.title('psf mask')
plt.show()

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(2,4,1)
temp = np.copy(sample)
temp[~psf_mask]=0
plt.imshow(temp,origin='lower',cmap=cmap,vmax=np.max(sample),vmin=np.min(sample))
plt.colorbar()
plt.title('PSF')

plt.subplot(2,4,2)
plt.hist(sample[psf_mask].ravel(),bins=np.linspace(975,2200,525))
plt.xlim([950,2200])
plt.ylim([0,500])
plt.title('data')
plt.ylabel('counts',fontsize=12)
plt.xlabel('pixel intensity',fontsize=12)

plt.subplot(2,4,3)
temp = np.copy(model)
temp[~psf_mask]=0
plt.imshow(temp,origin='lower',cmap=cmap,vmax=np.max(sample),vmin=np.min(sample))
plt.colorbar()
plt.title('PSF (model)')

plt.subplot(2,4,4)
plt.hist(model[psf_mask].ravel(),bins=np.linspace(975,2200,525))
plt.xlim([950,2200])
plt.ylim([0,500])
plt.title('model')
plt.ylabel('counts',fontsize=12)
plt.xlabel('pixel intensity',fontsize=12)


plt.subplot(2,4,5)
temp = np.copy(sample)
temp[psf_mask]=0
plt.imshow(temp,origin='lower',cmap=cmap,vmin=np.min(sample))
plt.colorbar()
plt.title('Background')

plt.subplot(2,4,6)
n,b,p = plt.hist(sample[~psf_mask].ravel(),bins=np.linspace(975,1050,75))
plt.xlim([980,1050])
plt.ylabel('counts',fontsize=12)
plt.xlabel('pixel intensity',fontsize=12)

plt.subplot(2,4,7)
temp = np.copy(model)
temp[psf_mask]=0
plt.imshow(temp,origin='lower',cmap=cmap,vmin=np.min(sample))
plt.colorbar()
plt.title('Background (model)')

plt.subplot(2,4,8)
plt.hist(model[~psf_mask].ravel(),bins=np.linspace(975,1500,525))
plt.xlim([980,1050])
plt.ylabel('counts',fontsize=12)
plt.xlabel('pixel intensity',fontsize=12)


plt.show()

In [ ]:
def estimate_background(image,fraction,p_scale,std_dev=7.5):
    '''
    Estimate background fluctiation from a psf image
    Current method needs as input the standard deviation of background. 
    This is needed to find the mask to get distribution of counts -- but result not very dependent on this input.
    '''
    minimized = fit_single_exposure(image,p_scale,std_dev)
    params = minimized.params
    img_model = plot_k_model(params,np.shape(image))
    
    #fraction of HLR of model
    fhl = fraction*(np.max(img_model)/2)
    psf_mask = img_model>=fhl
    
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    temp = np.copy(image)
    temp[psf_mask]=0
    plt.imshow(temp,origin='lower',cmap=cmap,vmin=np.min(image))
    plt.colorbar()
    plt.title('Background')

    background1D = image[~psf_mask].ravel()
    bins = np.linspace(min(background1D),max(background1D),max(background1D)-min(background1D))
    
    plt.subplot(1,2,2)
    plt.hist(background1D,bins=bins)
    plt.ylabel('counts',fontsize=12)
    plt.xlabel('pixel intensity',fontsize=12)

    plt.show()
    
    std = np.std(background1D)
    return std

In [ ]:
def estimate_background(image,p_scale,std_dev=7.5):
    '''
    Estimate background fluctiation from a psf image
    Current method needs as input the standard deviation of background. This is needed to find the mask to get distribution of counts -- but the result is not very dependent on this input.
    '''
    result = fit_single_exposure(image,p_scale,std_dev)
    result_P = result.params
    nx,ny = np.shape(image)
    x_ap = np.linspace(-nx/2,nx/2,nx)
    y_ap = np.linspace(-nx/2,nx/2,nx)
    X_ap,Y_ap = np.meshgrid(x_ap,y_ap)

    x0,y0 = int(result_P['offset_x']), int(result_P['offset_y'])
    r = 2.5*int(result_P['hlr']/p_scale)
    R = np.hypot(X_ap-x0,Y_ap-y0)
    psf_mask = R < r
    
    def FWHM(X,Y):
        half_max = max(Y) / 2.
        #find when function crosses line half_max (when sign of diff flips)
        #take the 'derivative' of signum(half_max - Y[])
        d = np.sign(half_max - np.array(Y[0:-1])) - np.sign(half_max - np.array(Y[1:]))
        #find the left and right most indexes
        left_idx = np.where(d > 0)[0][0]
        right_idx = np.where(d < 0)[0]
        fw = X[right_idx] - X[left_idx] #return the difference (full width)
        return np.sum(fw)/len(fw)
    
    background1D = image[~psf_mask].ravel()
    n,b,p = plt.hist(background1D,bins=np.linspace(975,1050,75))
    fwhm = FWHM(b,n)
    return fwhm/2.35